# Дипломный проект: Модель прогнозирования стоимости жилья для агентства недвижимости
## ОЧИСТКА ДАННЫХ

In [1]:
import pandas as pd
import numpy as np
import os

import json
import re
import statistics

In [2]:
data = pd.read_csv('data/data.csv', sep=',')
data_backup = data.copy()
data.head(3)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"


In [3]:
print('Размер датасета:', *data.shape)
print('Типы данных:', data.dtypes.value_counts())
display(data.describe())

Размер датасета: 377185 18
Типы данных: object    18
dtype: int64


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
count,337267,4181,342452,377183,270847,377185,103115,377151,377185,336608,377185,285903,377185,226470,24942,40311,310305,374704
unique,159,1,1280,337076,229,321009,1653,2026,297365,25405,4549,1184,39,348,24907,2,232944,43939
top,for sale,Yes,single-family home,Address Not Disclosed,2 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",yes,Houston,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,32137,3 Beds,FL,1.0,No MLS#,yes,NO MLS,"$225,000"
freq,156104,4181,92206,672,52466,7174,50356,24442,4204,11854,2141,53459,115449,67454,3,28793,24,1462


In [4]:
print('Общее число пропущенных значений:', data.isna().sum().sum(), end='\n\n')
print('Пропущ.значения по столбцам:', (data.isna().sum()*100/len(data)).round(5).sort_values(ascending=False), sep='\n\n')

Общее число пропущенных значений: 1869151

Пропущ.значения по столбцам:

private pool    98.89153
mls-id          93.38733
PrivatePool     89.31267
fireplace       72.66196
stories         39.95785
baths           28.19253
beds            24.20086
MlsId           17.73135
sqft            10.75785
status          10.58314
propertyType     9.20848
target           0.65777
city             0.00901
street           0.00053
zipcode          0.00000
schools          0.00000
state            0.00000
homeFacts        0.00000
dtype: float64


## Знакомство с данными data: очистка и предварительная работа
### Для начала приведем данные в порядок

## DATA ['PrivatePool'] vs DATA ['private pool']

In [5]:
def p_print(*args, **kwargs):
    '''Функция аналог print с определенными настройками end и sep.
    '''
    
    default_settings = {'end': '\n\n', 'sep': '\n'}
    kwargs = {**default_settings, **kwargs}
    print(*args, **kwargs)

In [6]:
# Посмотрим, какие значения встречаются в обоих полях
p_print("'private pool':", data['private pool'].value_counts())
p_print("'PrivatePool':", data['PrivatePool'].value_counts())

# Допустим, что эти поля дополняют данные об объектах. Объединим их
# сделаем перенос положительных значений из 'private pool'
yes_pool_index = data[data['private pool']=='Yes'].index
data['PrivatePool'][yes_pool_index] = 1

# Пусть 1 будет означать, что бассейн точно есть
data['PrivatePool'].replace(r'(yes|Yes)', 1, regex=True, inplace=True)
data['PrivatePool'].replace(np.NaN, 0, inplace=True)
p_print('Объединение данных:', data['PrivatePool'].value_counts())

#  Удалим ненужный признак
data.drop('private pool', axis=1, inplace=True)
data['PrivatePool'] = data['PrivatePool'].astype(np.int8)

'private pool':
Yes    4181
Name: private pool, dtype: int64

'PrivatePool':
yes    28793
Yes    11518
Name: PrivatePool, dtype: int64

Объединение данных:
0.0    332693
1.0     44492
Name: PrivatePool, dtype: int64



## DATA ['MLS-ID']

In [7]:
# Также два дублирующихся поля, причем одно из них имеет менее 18% пропусков, а другое более 90%.
# Удалим второе, предварительно сохранив уникальную информацию в первое поле.

mls_add_index = data[(data['MlsId'].isna()) & (~ data['mls-id'].isna())].index

data['MlsId'].iloc[mls_add_index] = data['mls-id'].iloc[mls_add_index]
data.drop(columns='mls-id', inplace=True)

p_print('Кол-во пропусков в столбце \'MlsId\':', (data['MlsId'].isna().sum()*100/len(data)).round(2))

# Но не является ли признак малоинформативным?
# Создадим функцию для определения малоинформативности, чтобы использовать ее и в дальнейшем

def check_informativeness(df_col=np.NaN, df=np.NaN):
    
    def calc_inform(col):
        max_val_ratio = round(col.value_counts().max()*100/col.shape[0], 3)
        nunique_ratio = round(col.nunique()*100/col.shape[0], 3)
        warning = 'is over 95%' if (max_val_ratio>95) or (nunique_ratio>95) else '-'
        
        result = pd.DataFrame({
            col.name: {
                'max_val_ratio': max_val_ratio, 
                'nunique_ratio': nunique_ratio,
                'warning': warning,
            }
        }).transpose()
        
        return result
    
    if df_col is not np.NaN:
        return calc_inform(df_col)
    
    if df is not np.NaN:
        df_dict = {}
        
        for col in df.columns:
            df_dict[col] = calc_inform(df[col]).iloc[0].to_dict()
    
        result = pd.DataFrame(df_dict).transpose()
        
    return result
    
    
p_print('Оценка малоинформативности:', end='\n')
display(check_informativeness(df_col=data['MlsId']))


C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_38248\3942647606.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MlsId'].iloc[mls_add_index] = data['mls-id'].iloc[mls_add_index]


Кол-во пропусков в столбце 'MlsId':
11.12

Оценка малоинформативности:


,max_val_ratio,nunique_ratio,warning
MlsId,0.007,66.019,-


In [8]:
# Признак 'MlsId' имеет допустимые показатели 
# Тем не менее из его уникальных значений можно извлечь лишь информацию о штате или городе, которая уже есть в других признаках
# Более того, данные по сути являются id из риелторской системы. Поэтому всё же удалим этот признак.
data.drop('MlsId', axis=1, inplace=True) 

## DATA ['fireplace']

In [9]:
fireplace = data_backup['fireplace'].copy().str.lower()
p_print('Топ-10 строк fireplace:', data['fireplace'].value_counts()[:10])

all_fireplace_phrases = fireplace.str.split(', ').explode()
p_print('Топ-10 фраз fireplace:', all_fireplace_phrases.value_counts()[:10])

# Сохраним основные фразы из признака о каминах в файл, чтобы удобнее было отбирать будущие категории
with open('data/other/all_fireplace_phrases.csv', 'wb') as file:
    all_fireplace_phrases.value_counts().to_csv(file)

Топ-10 строк fireplace:
yes               50356
Yes               20856
1                 14544
2                  2432
Not Applicable     1993
Fireplace           847
3                   564
Living Room         433
LOCATION            399
Wood Burning        311
Name: fireplace, dtype: int64

Топ-10 фраз fireplace:
yes                71212
1                  14546
2                   2432
not applicable      1993
ceiling fan         1318
gas logs            1272
living room         1153
wood burning        1108
walk-in closets     1042
fireplace            910
Name: fireplace, dtype: int64



In [10]:
# Заменим пустые значения на unknown, что будет означать: тип камина неизвестен
fireplace = fireplace.replace(np.NaN, 'unknown')

# Подгружаем заранее подготовленный файл с основными типами каминов из признака
fireplace_types = pd.read_csv('data/other/fireplace.csv')

fireplace_dict = fireplace_types.set_index('fireplace_type').to_dict()['fireplace_mask']
fireplace = fireplace.replace({fr'.*{key}.*': value for key, value in fireplace_dict.items()}, regex=True)

fireplace_mask = fireplace_types['fireplace_mask'].unique()
fireplace = fireplace.apply(lambda x: x if x in fireplace_mask else 'unknown')
p_print('Итоговые категории fireplace:', fireplace.value_counts())

data['fireplace_type'] = fireplace.astype('category')
data.drop(columns='fireplace', inplace=True)

Итоговые категории fireplace:
unknown        275466
yes             74269
1 frpl          18161
no               3780
2 frpl           2516
gas              1072
2 plus frpl       880
wood              826
decorative        215
Name: fireplace, dtype: int64



## DATA ['stories'], DATA['baths'], DATA['beds']

In [11]:
def replacement_helper(df_col, mode='full'):
    '''Функция делит значения признака на числовые и оставшиеся, сохраняя их в файлы.
    Таким образом это поможет найти "лишние" слова, от которых нужно очистить признаки до преобразования 
    '''
    
    df_col = df_col.str.lower().str.strip()
    
    df_col_num = df_col[df_col.str.contains(r'^[0-9,.]+$')==True]
    df_col_alpha = df_col[~df_col.isin(df_col_num)]
    
    df_col_num = df_col_num.value_counts()
    df_col_alpha = df_col_alpha.value_counts()
    
    dir_name = f'data/{df_col.name}'
    if not os.path.exists(dir_name): os.mkdir(dir_name)
    
    with open(f'{dir_name}/alpha_{df_col_alpha.name}.csv', mode='w', newline='') as file:
        df_col_alpha.reset_index().to_csv(file)
        #print(df_col_alpha.reset_index())

    # Выделим подстолбец с бессловесным описанием и сохраним его в отдельный файл
    with open(f'{dir_name}/num_{df_col_num.name}.csv', mode='w', newline='') as file:
        df_col_num.reset_index().to_csv(file)
    
    
    if mode=='full':
        p_print('alpha:', f'Длина столбца - {df_col_alpha.shape[0]}')
        p_print('num:', f'Длина столбца - {df_col_num.shape[0]}')
        p_print(f'Просмотрите файлы папки {dir_name} и преобразуйте столбец с помощью подходящих регулярных выражений, если необходимо.')
    else: 
        p_print(f'alpha: {df_col_alpha.shape[0]}, num: {df_col_num.shape[0]}')
    p_print('-------------------------------------------------------------------------')
    
    return df_col


In [12]:
stories = data_backup['stories'].copy()
stories = replacement_helper(stories)

# Просмотрев результаты выполнения replacement_helper, сделаем замены
stories_num_dict = {
    r'one': r'1', 
    r'two': r'2', 
    r'bi-?': r'2',
    r'three': r'3', 
    r'one and one half': r'1.5', 
    r'1 and 1 half': r'1.5',
    r'.*fourplex.*': '4', 
    r'.*duplex.*': '2', 
    r'tri': r'3',
    r'.*triplex.*': '3', 
    r'.*3plex.*': '3', 
    r'.*sixplex.*': '6', 
    r'.*quad.*': '4'
}

stories = stories.replace(stories_num_dict, regex=True)
stories = stories.replace(r'^.*?([\d+,.\s]+)[\s-]*(story|stories|level|levels).*$', r'\1', regex=True)
stories = stories.replace(r'.*(\d+)\s*(\+|or more).*', r'\1', regex=True)

# Доп.проверка:
stories = replacement_helper(stories, mode='short')

def try_float(str):
    try: 
        return float(str)
    except: 
        return np.NaN    
    
# Окончательно конвертируем признак в числовой   
stories = stories.apply(try_float)
data['stories_num'] = stories.astype(np.float16)
data.drop(columns='stories', inplace=True)

alpha:
Длина столбца - 176

num:
Длина столбца - 172

Просмотрите файлы папки data/stories и преобразуйте столбец с помощью подходящих регулярных выражений, если необходимо.

-------------------------------------------------------------------------

alpha: 69, num: 172

-------------------------------------------------------------------------



## DATA ['BATHS']

In [13]:
baths = data_backup['baths'].copy()
baths = replacement_helper(baths)

baths = baths.str.replace(r'[(ba|baths|bathrooms),.:\s]', '', regex=True)
baths = baths.str.replace('+', '', regex=False)

baths = baths.apply(try_float)
data['baths_num'] = baths.astype(np.float16)
data.drop(columns='baths', inplace=True)

alpha:
Длина столбца - 142

num:
Длина столбца - 85

Просмотрите файлы папки data/baths и преобразуйте столбец с помощью подходящих регулярных выражений, если необходимо.

-------------------------------------------------------------------------



## DATA ['beds']

In [14]:
beds = data_backup['beds'].copy()
beds = replacement_helper(beds)

# Сохраним на всякий случай данные о площади и удалим их из столбца о спальнях
acreage_from_beds = beds[beds.str.contains(r'(acre|acres|sqft)')==True]

beds = beds.replace(r'\b(bd|beds)\b$', '', regex=True)
beds = beds.replace(r'.*(\d+)\s(bedrooms).*', r'\1', regex=True)
beds = beds.replace(r'.*(acre|acres|sqft).*', np.NaN, regex=True)

beds = beds.apply(try_float)
data['beds_num'] = beds.astype(np.float16)
data.drop(columns='beds', inplace=True)
data['beds_num'] = data['beds_num']

alpha:
Длина столбца - 1124

num:
Длина столбца - 60

Просмотрите файлы папки data/beds и преобразуйте столбец с помощью подходящих регулярных выражений, если необходимо.

-------------------------------------------------------------------------



C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_38248\4183736315.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  acreage_from_beds = beds[beds.str.contains(r'(acre|acres|sqft)')==True]


## DATA ['sqft']

In [15]:
sqft = data_backup['sqft'].copy()
sqft = replacement_helper(sqft)

sqft = sqft.replace(r'.*([\d|\d+]?([,.]?\d+))\ssqft', r'\1', regex=True)
sqft = sqft.str.replace(r'(?<=\d)[.,](?=\d)', '.', regex=True)

sqft = sqft.apply(try_float)
data['sqft'] = sqft.astype(np.float16)

alpha:
Длина столбца - 13491

num:
Длина столбца - 11914

Просмотрите файлы папки data/sqft и преобразуйте столбец с помощью подходящих регулярных выражений, если необходимо.

-------------------------------------------------------------------------



## DATA ['homeFacts']

In [16]:
homefacts = data_backup['homeFacts'].copy()
# Небольшое исправление: 
homefacts[160470] = homefacts[160470].replace('"closet"', "'closet'")

def deserialize_homefacts(string):
    '''Функция позволяет десериализовать данные. Возвращает 0, если в процессе возникла ошибка
    '''
    string = string.replace("'atAGlanceFacts'", '"atAGlanceFacts"')
    string = re.sub(r":\sNone,", ': "",', string)
    string = re.sub(r"(?<=\{)'|'(?=\})", '"', string)
    string = re.sub(r"['|\"]([:|,]) ['|\"]", r'"\1 "', string)
    
    try:
        res = json.loads(string)
    except:
        res = 0
        
    return res


homefacts_dict = homefacts.apply(deserialize_homefacts)
homefacts_dict = homefacts_dict.apply(lambda x: x.get('atAGlanceFacts'))

# Преобразуем полученные данные в датафрейм:
hf_cols = [item['factLabel'] for item in homefacts_dict[0]]
hf_data = homefacts_dict.apply(lambda x: [item.get('factValue') for item in x])

# Формируем датафрейм, полученный из признака homeFacts
homefacts_df = pd.DataFrame(columns=hf_cols, data=hf_data.to_list())
display(homefacts_df)

,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
0,2019,,"Central A/C, Heat Pump",,,,$144
1,2019,,,,,5828 sqft,$159/sqft
2,1961,1967,Forced Air,Central,Attached Garage,"8,626 sqft",$965/sqft
3,2006,2006,Forced Air,Central,Detached Garage,"8,220 sqft",$371/sqft
4,,,,,,"10,019 sqft",
...,...,...,...,...,...,...,...
377180,1990,1990,Other,Central,2 spaces,"8,500 sqft",$311
377181,1924,,Radiant,,None,,$337/sqft
377182,1950,1950,Other,,2,"1,600 sqft",$458/sqft
377183,,,,,,,


In [17]:
homefacts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Year built      377185 non-null  object
 1   Remodeled year  377185 non-null  object
 2   Heating         377185 non-null  object
 3   Cooling         377185 non-null  object
 4   Parking         377185 non-null  object
 5   lotsize         377185 non-null  object
 6   Price/sqft      377185 non-null  object
dtypes: object(7)
memory usage: 20.1+ MB


In [18]:
homefacts_df.describe()

,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
count,377185,377185,377185,377185,377185,377185,377185
unique,230,154,1984,1445,3346,37393,6504
top,,,,Central,,,
freq,62374,226110,109332,158754,175420,61455,63738


# Поочередно разберем признаки homeFacts, чтобы дополонить данными датасет data

### homeFacts: признаки 'Year built', 'Remodeled Year'

In [19]:
homefacts_df['Year built'] = homefacts_df['Year built'].replace({'No Data': np.NaN, '': np.NaN}, regex=True).apply(float)
homefacts_df['Remodeled year'] = homefacts_df['Remodeled year'].replace({'': np.NaN}, regex=True).apply(float)

# Создание новых признаков:
# DATA ['age'] - возраст объекта, 
# DATA ['is_remodeled'] - был ли объект обновлён.

import datetime

current_year = datetime.datetime.now().year
data['age'] = current_year - homefacts_df['Year built']
data['is_remodeled'] = homefacts_df['Remodeled year'].apply(lambda x: 1 if x is not np.NaN else 0).astype(np.int8)

### homeFacts: признак 'Heating'

In [20]:
# Будем использовать все фразы из признака, чтобы выбрать наиболее популярные типы отопления
heating = homefacts_df['Heating'].str.lower()
print('Топ-10 фраз Heating:')
display(heating.str.split(', ').explode().value_counts()[:10])

# Основные тип ы отопления подгрузим из подготовленного файла
heating_types = pd.read_csv('data/other/heating.csv', usecols=[0, 1])
print('Фрагмент файла для присвоения категорий:')
display(heating_types.head())

Топ-10 фраз Heating:


forced air          139497
                    111541
other                30148
electric             16413
central              12644
heat pump            12379
gas                  11531
central air           9867
central electric      8768
no data               8611
Name: Heating, dtype: int64

Фрагмент файла для присвоения категорий:


,heating_mask,heating_type
0,air,air
1,electric,electric
2,heat pump,heat pump
3,heat pump,pump
4,gas,propane


In [21]:
heating_dict = heating_types.set_index('heating_type').to_dict()['heating_mask']
heating = heating.replace({fr'.*{key.strip()}.*': value for key, value in heating_dict.items()}, regex=True)

heating_mask = list(heating_types['heating_mask'].unique())


def to_categories(split, mask):
    
    for item in split: 
        if item in mask:
            return item
    return 'other'


heating = heating.str.split(', ').apply(lambda x: to_categories(x, heating_mask))

print('Полученные категории heating')
display(heating.value_counts())

data['heating_type'] = heating.astype('category')

Полученные категории heating


other                      158948
air                        153274
electric                    28853
gas                         13877
heat pump                   10177
baseboard                    8330
radiant                      1745
furnace/stove/fireplace      1242
oil                           271
zoned                         213
water                         199
steam                          56
Name: Heating, dtype: int64

### homeFacts: признак 'Cooling'

In [22]:
cooling = homefacts_df['Cooling'].str.lower().str.split(', ').explode()
cooling.value_counts()[:10]

print('Топ-10 фраз Cooling:')
display(cooling.value_counts()[:10])

# Много информации, отсылающей к системе отопления. Пока не будем использовать в датасете этот столбец.

Топ-10 фраз Cooling:


central                   162224
                          124041
central air                18296
no data                    10616
has cooling                 9730
central electric            7976
none                        7441
wall                        4846
central gas                 4600
central a/c (electric)      3907
Name: Cooling, dtype: int64

### homeFacts: признак 'Parking'

проработать пустые строки вида '' заменить их на np.NaN в data и homeFacts

In [23]:
parking = homefacts_df['Parking'].str.lower()
print('Топ-10 фраз Parking:')
display(parking.str.split(', ').explode().value_counts()[:10])

parking_types = pd.read_csv('data/other/parking.csv', sep=', ', engine='python')
print('Фрагмент файла для присвоения категорий:')
display(parking_types.head())

Топ-10 фраз Parking:


                   175430
attached garage     80465
2 spaces            28063
detached garage     17019
1 space             14253
no data             13334
carport             13175
off street           8181
3 spaces             4724
on street            3370
Name: Parking, dtype: int64

Фрагмент файла для присвоения категорий:


,parking_mask,parking_type
0,no parking,none
1,garage,opener
2,garage,rv gate
3,attached,attached
4,detached,detached


In [24]:

reg_parking_dict = {
    r'.*([1-4])(?!\d).*(spaces|space).*': r'\1 spaces',
    r'.*(spaces|space).*([1-4])(?!\d).*': r'\1 spaces',
    r'.*([5-9])(?!\d).*(spaces|space).*': r'5 plus spaces', # означает от 5 до 9 м/м
    r'.*(spaces|space).*([5-9])(?!\d).*': r'5 plus spaces',
    r'.*([1-9][0-9]+).*(spaces|space).*': r'10 plus spaces', 
    r'.*(spaces|space).*([1-9][0-9]+).*': r'10 plus spaces'
}

# Будем считать, что одинокая цифра в строке означает кол-во машиномест:
parking = parking.replace(r'^(\d+)$', r'\1 spaces', regex=True)

parking = parking.replace(reg_parking_dict, regex=True)

main_parking_dict = parking_types.set_index('parking_type').to_dict()['parking_mask']
parking = parking.replace({fr'.*{key.strip()}.*': fr'{value}' for key, value in main_parking_dict.items()}, regex=True)

parking = parking.replace(r'garage', 'garage', regex=True)

parking_mask = list(parking_types['parking_mask'].unique())
parking = parking.str.split(', ').apply(lambda x: to_categories(x, parking_mask))

print('Топ-10 фраз parking:')
display(parking.value_counts())

data['parking_type'] = parking.astype('category')

Топ-10 фраз parking:


other             194987
attached           81968
2 spaces           31170
1 spaces           17340
detached           14877
carport             8745
off street          6858
3 spaces            5213
4 spaces            3590
garage              3388
5 plus spaces       2707
no parking          2415
on street           1901
driveway            1253
slab                 509
10 plus spaces       238
oversized             26
Name: Parking, dtype: int64

### homeFacts: признаки 'lotsize' и 'Price/sqft'

In [25]:
homefacts_df['lotsize'].str.lower().value_counts()
# Мы ранее уже обработали признак с площадью объекта, текущий пока использовать не будем

                  61455
—                 25251
no data            5330
-- sqft lot        3819
0.26 acres         3140
                  ...  
4,396 sqft            1
8,661 sqft            1
5,591 sqft            1
5,716 sq. ft.         1
7,084 sqft lot        1
Name: lotsize, Length: 36608, dtype: int64

In [26]:
homefacts_df['Price/sqft'].str.lower().value_counts()
# В этом признаке есть непосредственная связь с таргетом, поэтому его тоже не будем использовать

               63738
no data         1241
$1/sqft          974
no info          954
$125/sqft        797
               ...  
$3,077/sqft        1
$2,244/sqft        1
$1,457             1
$6,141             1
$2,032             1
Name: Price/sqft, Length: 6504, dtype: int64

In [27]:
data.drop(columns='homeFacts', inplace=True)

## DATA ['schools']

In [28]:
schools = data_backup['schools'].copy()


def deserialize_schools(string):
    '''Функция позволяет десериализовать данные. Возвращает 0, если в процессе возникла ошибка
    '''
    string = re.sub(r"(?<=[,\s|\{|\[])'|'(?=[:|,\s|\]])", '"', string)
    string = re.sub(r'(?<=\w\s)"|"(?=\s\w)', "'", string)
    string = string.replace(', None', ', ""')

    try:
        res = json.loads(string)[0]
    except:
        res = 0
    
    return res

# Формируем series из словарей, содержащих информацию о школах для всех объектов датасета
schools_dict = schools.apply(deserialize_schools) 
print('Проверим, сколько строк не удалось сериализовать:')
print(schools_dict[schools_dict==0].shape[0])

Проверим, сколько строк не удалось сериализовать:
0


In [29]:
def rating_stat(schools_dict):
    '''Функция для вычисления среднего рейтинга ближайших школ
    '''
    rating = schools_dict['rating']
    rating = [re.sub(r'/.*', '', item) for item in rating]
    rating = [int(item) for item in rating if item.isnumeric()]
    
    return round(statistics.mean(rating), 2) if len(rating)>0 else np.NaN


def num_schools(schools_dict):
    '''Функция для вычисления количества ближайших школ
    '''
    names = schools_dict['name']
    return len(names) if names!=[] else np.NaN


def distance_stat(schools_dict):
    '''Функция для вычисления показателей расстояния до ближайших школ:
        среднего, максимального, минимального
    '''
    distance = schools_dict['data']['Distance']
    distance = [float(item.replace('mi', '')) for item in distance]
    distance = [item for item in distance if item is not np.NaN]
    
    if len(distance) == 0: return np.NaN
    
    mean_distance = round(statistics.mean(distance), 2)
    
    return mean_distance


mean_school_rating = schools_dict.apply(rating_stat)
schools_number = schools_dict.apply(num_schools)
mean_school_distance = schools_dict.apply(distance_stat)

data['mean_school_rating'] = mean_school_rating
data['schools_number'] = schools_number
data['mean_school_distance'] = mean_school_distance

In [30]:
import itertools

grades = pd.Series([row['data']['Grades'] for row in schools_dict])
grades = grades.apply(lambda lst: [re.split(r'[-|to|–|,]', elem) for elem in lst])
grades = grades.apply(lambda lst: list(set(itertools.chain.from_iterable(lst))))
grades = grades.apply(lambda lst: [elem.lower().strip() for elem in lst])

grades_pk = grades.apply(lambda lst: 1 if ('pk' in lst) or ('presch' in lst) else 0)
grades_k = grades.apply(lambda lst: 1 if 'k' in lst else 0)

num_grades = grades.apply(lambda lst: list(filter(lambda x: x.isnumeric(), lst)))
num_grades = num_grades.apply(lambda lst: [int(elem) for elem in lst])

min_grade = num_grades.apply(lambda lst: min(lst) if len(lst)>0 else np.NaN)
max_grade = num_grades.apply(lambda lst: max(lst) if len(lst)>0 else np.NaN)

data['grades_pk'] = grades_pk
data['grades_k'] = grades_k

data['min_grade'] = min_grade
data['max_grade'] = max_grade

In [31]:
data.drop('schools', axis=1, inplace=True)

# STATE, CITY, STREET, ZIPCODE
- Рассмотреть возможность загрузки достоверных списков штатов, городов, улиц.
- Проанализировать повторяющиеся индексы - не ошибка ли это?
- Кодировка или удаление?


## DATA ['state']

In [32]:
state = data_backup['state'].copy()
p_print('Список штатов из датасета:', (state.unique()))

state_data = pd.read_csv('data/other/states_zipcodes.csv')
#state_data['State'] = state_data['State'].str.strip()
print('Фрагмент проверочного файла Штаты/зипкоды:')
display(state_data.head(2))

drop_states = []

for item in state.unique():
    if item not in list(state_data['Short Name']): drop_states.append(item)
    
p_print('Несуществующие штаты:', drop_states)
#Округ Колумбию оставим, остальные преобразуем в np.NaN
drop_states.remove('DC')
state[state.isin(drop_states)] = np.NaN
p_print('Количество пустых строк state:', state.isna().sum())

data['state'] = state

Список штатов из датасета:
['NC' 'WA' 'CA' 'TX' 'FL' 'PA' 'TN' 'IA' 'NY' 'OR' 'DC' 'NV' 'AZ' 'GA'
 'IL' 'NJ' 'MA' 'OH' 'IN' 'UT' 'MI' 'VT' 'MD' 'CO' 'VA' 'KY' 'MO' 'WI'
 'ME' 'MS' 'OK' 'SC' 'MT' 'DE' 'Fl' 'BA' 'AL' 'OT' 'OS']

Фрагмент проверочного файла Штаты/зипкоды:


,State,Short Name,Capital City,ZIP Codes
0,Alabama,AL,Montgomery,35004;36925
1,Alaska,AK,Juneau,99501;99950


Несуществующие штаты:
['DC', 'Fl', 'BA', 'OT', 'OS']

Количество пустых строк state:
4



In [33]:
# Дополним проверочную таблицу
state_data.loc[state_data.shape[0]] = ['District of Columbia', 'DC', '', '20001;20599']

## DATA ['zipcode']

In [34]:
zipcode = data['zipcode'].copy()

# В строках встречаются зипкоды нестандартного вида. Исправим это 
data['zipcode'] = data['zipcode'].replace(r'-.*', '', regex=True)

# Заменим колонку с зипом в проверочной табл. на две - с границами
zips = state_data['ZIP Codes'].str.split(';')
state_data['ZIP min'] = [zip_min for [zip_min, _] in zips]
state_data['ZIP max'] = [zip_max for [_, zip_max] in zips]
state_data.drop(columns='ZIP Codes', inplace=True)

print('Новое представление проверочной таблицы:')
display(state_data.head(1))



Новое представление проверочной таблицы:


,State,Short Name,Capital City,ZIP min,ZIP max
0,Alabama,AL,Montgomery,35004,36925


In [35]:
# Составим словарь для проверки границ зипкодов по штату
zip_dict = state_data[['Short Name', 'ZIP min', 'ZIP max']].set_index('Short Name').T.to_dict(orient='list')
p_print('Словарь {штат: границы зипкода}:', zip_dict)

# Перенесем границы зипкодов в основной датасет
data['ZIP_bounds'] = data['state'].apply(lambda x: zip_dict.get(x, 'unknown state'))
p_print('Неопределенности при переносе границ зипкода:', data['ZIP_bounds'][data['ZIP_bounds']=='unknown state'].shape[0])

# Проверим соответствуют ли индексы из датасета необходимым границам:
def zip_state_check_func(series):
    
    try:
        return True if (int(series[0]) >= int(series[1][0])) & (int(series[0]) <= int(series[1][1])) else False
    except:
        return 'error'

data['zip_state_check'] =  data[['zipcode', 'ZIP_bounds']].apply(zip_state_check_func, axis=1)
print('Несоответствие штатов индексам в датасете:')
print('Кол-во:', data['zip_state_check'].value_counts()[False])
print('Примеры:')
display(data[data['zip_state_check'] == False][['state', 'zipcode', 'ZIP_bounds', 'zip_state_check']].head())

print('Ошибки в соответствии индексов штатам:')
display(data[data['zip_state_check']=='error'][['state', 'zipcode', 'ZIP_bounds']])

Словарь {штат: границы зипкода}:
{'AL': ['35004', '36925'], 'AK': ['99501', '99950'], 'AZ': ['85001', '86556'], 'AR': ['71601', '72959'], 'CA': ['90001', '96162'], 'CO': ['80001', '81658'], 'CT': ['06001', '06928'], 'DE': ['19701', '19980'], 'FL': ['32003', '34997'], 'GA': ['30002', '39901'], 'HI': ['96701', '96898'], 'ID': ['83201', '83888'], 'IL': ['60001', '62999'], 'IN': ['46001', '47997'], 'IA': ['50001', '52809'], 'KS': ['66002', '67954'], 'KY': ['40003', '42788'], 'LA': ['70001', '71497'], 'ME': ['03901', '04992'], 'MD': ['20601', '21930'], 'MA': ['01001', '05544'], 'MI': ['48001', '49971'], 'MN': ['55001', '56763'], 'MS': ['38601', '39776'], 'MO': ['63001', '65899'], 'MT': ['59001', '59937'], 'NE': ['68001', '69367'], 'NV': ['88901', '89883'], 'NH': ['03031', '03897'], 'NJ': ['07001', '08989'], 'NM': ['87001', '88441'], 'NY': ['00501', '14925'], 'NC': ['27006', '28909'], 'ND': ['58001', '58856'], 'OH': ['43001', '45999'], 'OK': ['73001', '74966'], 'OR': ['97001', '97920'], 'PA'

,state,zipcode,ZIP_bounds,zip_state_check
24403,WA,90109,"[98001, 99403]",False
30261,CA,0,"[90001, 96162]",False
50596,FL,3316,"[32003, 34997]",False
54590,TN,17238,"[37010, 38589]",False
83522,NY,0,"[00501, 14925]",False


Ошибки в соответствии индексов штатам:


,state,zipcode,ZIP_bounds
113694,NaN,33321,unknown state
172273,NaN,33179,unknown state
193466,NaN,11210,unknown state
231282,CA,,"[90001, 96162]"
235207,FL,,"[32003, 34997]"
308229,NaN,00000,unknown state


Несоответствия будем устранять далее после обработки признаков города и адреса.

## DATA['city']

In [36]:
city = data_backup['city'].copy().str.lower()
p_print('Число уникальных городов в датасете:', city.nunique())

Число уникальных городов в датасете:
1909



In [37]:
from difflib import SequenceMatcher

def find_matching_strings(original_string, strings, similarity_threshold):
    '''Функция для поиска похожих строк, которая поможет найти города, в написании которых содержится ошибка.
    Возвращает список, где ключевому названию будут сопоставлены альтернативные вариантыб включая изначальный.
    Поиск осуществляется согласно коэффициенту similarity_threshold.
    '''
    
    matching_strings = []
    for string in strings:
        similarity_ratio = SequenceMatcher(None, original_string, string).ratio()
        if similarity_ratio >= similarity_threshold:
            matching_strings.append(string)
    return matching_strings


similarity_threshold = 0.9
unique_cities = list(city.unique())
unique_cities.remove(np.NaN)

city_dict = {}

for city_name in unique_cities:
    unique_cities.remove(city_name)
    matching_strings = find_matching_strings(city_name, unique_cities, similarity_threshold)
    if len(set(matching_strings)) > 0: 
        city_dict[city_name] = list(set(matching_strings))
        
        
cities_file = pd.Series(city_dict).explode()

with open('data/other/city.csv', 'w', newline='') as file:
    cities_file.to_csv(file)
    
# Далее вручную обработаем сохраненный файл, указывая в третьей колонке номер 1 или 2 в соответствии с предпочитаемым вариантом написания города
# 0 - если оба варианта уникальны.

In [38]:
city_correct = pd.read_csv('data/other/city_correct.csv', names=['name_1', 'name_2', 'var'])
cc = city_correct[city_correct['var'] > 0].apply(lambda x: [x['name_1'], x['name_2']] if x['var']==1 else {x['name_2'], x['name_1']}, axis=1)
cc = list(cc)[1:]
city_correct_dict = {key: value for [key, value] in cc}

city = city.replace(city_correct_dict, regex=True)
city.nunique()
# Получилось очень мало замен :(

data['city'] = city

## DATA ['street]

In [39]:
street = data_backup['street'].copy()
street.value_counts()[:5].index
# Есть строки без указания адреса, позже они попадут в список неопределнных типов

Index(['Address Not Disclosed', 'Undisclosed Address', '(undisclosed Address)',
       'Address Not Available', 'Unknown Address'],
      dtype='object')

In [40]:
def clean_streets(string):
    '''Очистка строк с названиями улиц от знаков препинания и фрагментов с числами'''
    
    if string is np.NaN: return np.NaN
    
    string = re.sub(r'[,|.|#]', '', string.lower())
    string = re.sub(r':', ' ', string)
    string = re.sub(r'\b\w*\d\w*\b|#\d|\d#', '', string)
    
    return string


def last_word_street_type(series):
    '''Возвращает объект Series, в котором содержатся последние слова (окончания) из наименования улиц и их частота, 
    без учета знаков препинания и фрагментов с числами.
    Исходим из предположения, что среди самых частых окончаний окажутся преимущественно типы улиц.'''
    
    temp_street = series.apply(clean_streets)
    series_split = temp_street.str.split()
    
    return series_split.str.get(-1).value_counts()


# Задаем ограничитель вывода предполагаемых типов улиц:
n = 25

print('Ключевые типы улиц по последнему слову в строке:')
street_last_words = list(last_word_street_type(street).index[:n])
print(street_last_words, end='\n\n')

def most_frequent_words(series):
    '''Возвращает объект Series, в котором содержится частота встречаемости каждого слова из названия улиц, 
    без учета знаков препинания и фрагментов с числами.
    Исходим из предположения, что среди самых частых слов окажутся преимущественно типы улиц.
    '''
    common_list = []
    temp_street = series.apply(clean_streets)
    series_split = temp_street.str.split()
    #series_split = series.str.replace(r'[,|.]', '', regex=True).str.replace(r'\b\w*\d\w*\b|#\d|\d#', '', regex=True).str.split()

    for split in series_split: 
        if split is np.NaN: continue
        common_list.extend(split)
    
    return pd.Series(common_list).value_counts()


print('Ключевые типы улиц по частоте слов в столбце:')
street_most_freq = list(most_frequent_words(street).index[:n])
print(street_most_freq, end='\n\n')

# Объединим списки n предполагаемых типов, найденные двумя способами
print(f'Общий список из {n} предполагаемых типов улиц:')
main_street_types = list(set(street_last_words+street_most_freq))
print(main_street_types)


Ключевые типы улиц по последнему слову в строке:
['st', 'dr', 'ave', 'rd', 'ln', 'ct', 'apt', 'blvd', 'way', 'pl', 'unit', 'cir', 'ter', 'mls', 'ne', 's', 'nw', 'n', 'trl', 'se', 'sw', 'lot', 'e', 'w', 'a']

Ключевые типы улиц по частоте слов в столбце:
['st', 'dr', 'ave', 'rd', 'ln', 'n', 'w', 'ct', 's', 'sw', 'e', 'nw', 'apt', 'blvd', 'ne', 'plan', 'unit', 'pl', 'way', 'cir', 'in', 'se', 'ter', 'mls', 'park']

Общий список из 25 предполагаемых типов улиц:
['st', 'ter', 'apt', 'in', 'sw', 'trl', 'ln', 'ave', 'pl', 'dr', 'cir', 's', 'rd', 'ct', 'plan', 'se', 'ne', 'n', 'e', 'mls', 'w', 'unit', 'nw', 'lot', 'way', 'blvd', 'park', 'a']


In [41]:
# Рассмотрим примеры из столбца с подобранными типами улиц, чтобы избавиться от неподходящих.
print('Примеры употребления основных типов улиц.', 'Найдите "мусорные" типы:', sep='\n', end='\n\n')

cleaned_street = street.apply(clean_streets)

for elem in main_street_types:
    street_subseries = street[cleaned_street.str.contains(fr"\b{elem}\b")==True]
    print(f'{street_subseries.iloc[np.random.randint(len(street_subseries))]}, тип: {elem}')
       
print('\n\n')

# Составим список 'мусорных' типов улиц для удаления из основного списка 
garbage_street_types = ['mls', 'apt', 's', 'w', 'n', 'e', 'ne', 'se', 'sw', 'nw', '#', '#1', '#2', 'a', 'b', 'unit', 'in', 'lot']
main_street_types = list(set(main_street_types).difference(set(garbage_street_types)))
print('Очищенный основной список типов:', main_street_types, end='\n\n', sep='\n')


def street_type_determine(string):
    '''Проверяет, содержится ли метка основного типа улиц в строке. Возвращает метку класса или Nan, если метка не найдена.
    '''
    if string is np.NaN: return np.NaN
    
    # Будем использовать сплит строки и определять тип по первому совпадению из него с элементами основных типов.
    str_split = re.split(r'[\s|.|,]', string.lower())
    str_split = list(filter(lambda x: x!='', str_split))
    
    for item in str_split[::-1]:
        if item in main_street_types: return item
    
    return 'other'

# Найдем названия улиц, которые не были отнесены к любому основному типу, и поищем основные типы среди них
notype_street = street[street.apply(street_type_determine) == 'other']
print('Дополнительный список типов улиц:', end='\n')
print(list(set(most_frequent_words(notype_street).index[:25]).difference(set(main_street_types))))

Примеры употребления основных типов улиц.
Найдите "мусорные" типы:

1305 Holloway St , тип: st
2845 Mcgill Ter NW, тип: ter
14740 E Kentucky Dr APT 721, тип: apt
The Landon Plan in American West Fox Hill Estates, тип: in
8437 SW 137th Ave #8437 , тип: sw
5578 Tiger Trl, тип: trl
20919 Burnt Amber Ln , тип: ln
4201 Ocean Ave, тип: ave
5945 Atlas Pl SW, тип: pl
707 Ginger Lake Dr Unit 136, тип: dr
11749 Fitchwood Cir, тип: cir
2154 S Balboa Plz, тип: s
Chain Bridge Rd NW, тип: rd
17708 NW Connett Meadow Ct, тип: ct
Oasis Plan in K-Bar Ranch, тип: plan
925 Potomac Ave SE, тип: se
620 Peachtree St NE #701, тип: ne
4810 N Ferdinand St, тип: n
2405 E 19th Ave # 1-2, тип: e
MLS #: CORC5938590, тип: mls
3152 W Calavar Rd, тип: w
9050 W Warm Springs Rd UNIT 2054, тип: unit
12878 NW Milazzo Ln, тип: nw
Lake Bend Ct Lot 24, тип: lot
5406 Matanzas Way, тип: way
7967 Hampton Park Blvd E, тип: blvd
7768 Holly Park Ct NW, тип: park
501 E 37th St #A, тип: a



Очищенный основной список типов:
['plan',

In [42]:
# Доработаем список ключевых типов улиц
main_street_types.extend(['path', 'pkwy', 'loop', 'hwy', 'cv', 'park', 'pass', 'rdg', 'pike', 'creek', 'clf'])
# Унифицируем некоторые типы
replace_dict = {'avenue': 'ave', 'road': 'rd', 'cove': 'cv', 'court': 'ct', 'place': 'pl', 'drive': 'dr', 'highway': 'hwy', 'lane': 'ln', 'cliff': 'clf', 'roadway': 'rdg'}
street = street.replace(replace_dict, regex=True)

# Сохраним полученные данные в новый столбец датасета
data['street_type'] = data['street'].apply(street_type_determine)
data.drop('street', axis=1, inplace=True)

## PropertyType

In [43]:
property_type = data_backup['propertyType'].copy()
property_type = property_type.str.lower().str.replace('-',' ')

# Подгрузим заранее подготовленные дубликаты propertyType
pt_df = pd.read_csv('data/other/property_type.csv', sep=': ', names=['property_type', 'property_mask'], engine='python')
pt_df.replace(r"[',]", '', regex=True, inplace=True)

# Составим из них словарь для замены
pt_replace_dict = pt_df.set_index('property_type').to_dict()['property_mask']
property_type = property_type.replace(pt_replace_dict, regex=True)

# Разобьем значения propertyType на список, чтобы можно было определить основной тип по первому вхождению
property_type_split = property_type.apply(lambda x: re.split(r'[/|,]', x) if x is not np.NaN else np.NaN)

main_property_types = ['apartment', 'condo', 'co op', 'single family', 'townhome', 'cape cod', 'colonial', 'contemporary', 'cottage', 'craftsman',
 'greek revival', 'farmhouse', 'french country', 'mediterranean', 'midcentury', 'ranch', 'split level', 'tudor', 'victorian', 'european houses style',
 'log home', 'manufactured home', 'cabin', 'land', 'multi family', 'traditional', 'transitional', 'bungalow', 'high rise', '1 story', '2 stories', 'penthouse', 'duplex', 'fourplex',
 'garden home', 'other']


def property_type_determine(str_split):
    
    if str_split is np.NaN: return np.NaN
    
    for elem in str_split:
        elem = re.sub(r'\(.*\)', '', elem)
        elem = elem.strip()
        if elem in main_property_types: return elem
    
    return 'other'


data['property_type'] = property_type_split.apply(property_type_determine)
data.drop('propertyType', axis=1, inplace=True)
p_print('Число основный категорий propertyType:', len(main_property_types))
print('Топ-10 категорий:')
display(data['property_type'].value_counts()[:10])

Число основный категорий propertyType:
36

Топ-10 категорий:


single family        189693
condo                 51461
land                  31486
townhome              18552
multi family          12218
traditional            7930
other                  5830
contemporary           3632
manufactured home      3544
co op                  3306
Name: property_type, dtype: int64

## DATA ['status]

In [44]:
status = data_backup['status'].copy()
status = status.str.lower()

# Подгрузим заранее подготовленный файл с категориями статуса
status_df = pd.read_csv('data/other/status.csv')
main_status = status_df['main status']

def status_type_determine(string):
    if string is np.NaN: return np.NaN
    string = re.sub(r'\s*/\s*', ' ', string)
    string = re.sub(r'(?<=coming soon).*', '', string)
    string = re.sub(r'\s+', ' ', string).strip().replace('-', ' ')
    
    if string in list(main_status): return status_df['mask'][list(main_status).index(string)]
    
    return 'other'
        
    
data['status_type'] = status.apply(status_type_determine)
data.drop('status', axis=1, inplace=True)
print('Статусы преобразованы согласно категориям')
print('Топ-10 категорий:')
display(data['status_type'].value_counts()[:10])

Статусы преобразованы согласно категориям
Топ-10 категорий:


activated                 304778
foreclosed                  7228
new construction            6165
pending                     4917
pre foreclosure             3679
under contract showing      3386
other                       2446
active auction              1472
contingency                  633
price change                 563
Name: status_type, dtype: int64

In [45]:
# Сохраняем итоговый датасет для дальнейшей работы

data.drop(['ZIP_bounds', 'zip_state_check'], axis=1, inplace=True)
data_2backup = data.copy()
data_2backup.to_csv('data/data_2backup.csv')